In [1]:
from operator import imod
import sys
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import LearningRateScheduler, LambdaCallback
from tensorflow.keras.models import load_model
from pathlib import Path
import platform
sys.path.append(os.path.abspath("../../src"))
from dual_network import dual_network, DN_INPUT_SHAPE
from auto_play import AutoPlay
from const import HISTORY_DIR, MODEL_DIR
from train_network import train_network, update_best_player, load_data

def train_network(old_model_path, new_model_path, batch_size, epochs = 100, shape=DN_INPUT_SHAPE, 
        history_path = sorted(Path(HISTORY_DIR).glob('*.history'))[-1]):
    if platform.system() == "Darwin":
        from tensorflow.python.compiler.mlcompute import mlcompute
        mlcompute.set_mlc_device(device_name="gpu")
    else:
        physical_devices = tf.config.list_physical_devices('GPU')
        if len(physical_devices) > 0:
            for device in physical_devices:
                tf.config.experimental.set_memory_growth(device, True)
                print('{} memory growth: {}'.format(device, tf.config.experimental.get_memory_growth(device)))
        else:
            print("Not enough GPU hardware devices available")
    history = load_data(history_path)
    input, policies, values = zip(*history)

    height, width, channel = shape
    input = np.array(input)
    input = input.transpose(0, 2, 3, 1)
    input = input.reshape(len(input), height, width, channel)   
    policies = np.array(policies)
    values = np.array(values)

    model = load_model(old_model_path)

    model.compile(loss=['categorical_crossentropy', 'mse'], optimizer='SGD')

    def step_decay(epoch):
        x = 0.001
        if epoch >= 50: x = 0.0005
        if epoch >= 80: x = 0.00025
        return x
    lr_decay = LearningRateScheduler(step_decay)

    print_callback = LambdaCallback(
        on_epoch_begin=lambda epoch,logs:
                print('\rTrain {}/{}'.format(epoch + 1,epochs), end=''))
    
    model.fit(input, [policies, values], batch_size=batch_size, epochs=epochs,
            verbose=0, callbacks=[lr_decay, print_callback])
    print('')

    model.save(new_model_path)

    K.clear_session()
    del model

ImportError: cannot import name 'DN_INPUT_SHAPE' from 'game' (/Users/daiki/SimpleCardGame/src/game.py)

In [1]:
from auto_play_case2 import AutoPlayCase2
current_dir = %pwd
best_model = current_dir+"/case2_best.h5"
latest_model = current_dir+"/case2_latest.h5"
dual_network(best_model, (9, 5, 16))


train_network(best_model, latest_model)

eval_best_player = AutoPlayCase2()
eval_best_player.action1("pv_ismcts", latest_model)
eval_best_player.action2("ismcts")
eval_best_player.calc_win_rate(5, 100, 0.0, 'Evaluate Best Player vs ismcts', True)



PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True
Train 0 ====================
Evaluate New Network vs Old
Completed 100/100Win rate : 0.74
Change BestPlayer
Evaluate Best Player vs ismcts
Completed 100/100Win rate : 0.08000000000000002
Train 1 ====================
Self Play
Completed 36/2000

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True
Train 0 ====================
Self Play
Completed 2000/2000PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True
Train 100/100
Evaluate New Network vs Old
Completed 100/100Win rate : 0.66
Change BestPlayer
Evaluate Best Player vs ismcts
Completed 100/100Win rate : 0.08000000000000002
Train 1 ====================
Self Play
Completed 8/2000

Train 4 ====================
Self Play
Completed 2000/2000PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True
Train 100/100
Evaluate New Network vs Old
Completed 100/100Win rate : 0.46


PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True


NameError: name 'i' is not defined